In [ ]:
!pip install langchain langchain-community tiktoken chromadb openai pypdf sentence-transformers --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00


In [ ]:
!wget https://constitutioncenter.org/media/files/constitution.pdf

--2024-09-20 03:32:39--  https://constitutioncenter.org/media/files/constitution.pdf
Resolving constitutioncenter.org (constitutioncenter.org)... 104.22.22.181, 172.67.42.106, 104.22.23.181, ...
Connecting to constitutioncenter.org (constitutioncenter.org)|104.22.22.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 413949 (404K) [application/pdf]
Saving to: ‘constitution.pdf’

constitution.pdf    100%[===================>] 404.25K  --.-KB/s    in 0.02s   

2024-09-20 03:32:40 (16.0 MB/s) - ‘constitution.pdf’ saved [413949/413949]



In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = '<YOUR API KEY HERE>'

# Step 1: Load the PDF and split it into manageable chunks
def load_and_split_pdf(pdf_file_path):
    loader = PyPDFLoader(pdf_file_path)
    documents = loader.load()

    # Split the document into smaller chunks for better processing
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)
    return texts

# Step 2: Store embeddings in ChromaDB
def store_embeddings_in_chromadb(documents):
    embeddings = OpenAIEmbeddings()  # You can also use SentenceTransformer embeddings if preferred
    vectorstore = Chroma.from_documents(documents, embeddings)
    return vectorstore

# Step 3: Setup a retriever and chain for querying and context retrieval
def create_retrieval_chain(vectorstore):
    llm = OpenAI(model="gpt-3.5-turbo-instruct",openai_api_key=os.environ["OPENAI_API_KEY"])
    retrieval_qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Optionally use "map_reduce" or others for more advanced chaining
        retriever=vectorstore.as_retriever()
    )
    return retrieval_qa

# Main Program
if __name__ == "__main__":
    # Sample PDF file path
    pdf_file_path = 'constitution.pdf'

    # Step 1: Load and split the PDF into chunks
    print("Loading and splitting PDF...")
    documents = load_and_split_pdf(pdf_file_path)

    # Step 2: Store document embeddings in ChromaDB
    print("Storing embeddings in ChromaDB...")
    vectorstore = store_embeddings_in_chromadb(documents)

    # Step 3: Create a retrieval-based chain with LLM
    retrieval_qa = create_retrieval_chain(vectorstore)

    # Step 4: User query input
    user_query = input("Enter your query: ")

    # Step 5: Retrieve context and generate answer using LLM
    answer = retrieval_qa.run(user_query)
    print(f"LLM Response: {answer}")


Loading and splitting PDF...
Storing embeddings in ChromaDB...
Enter your query: What amendment protects the right to bear arms?
LLM Response:  Amendment II.
